# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install python-twitter
import twitter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
import numpy as np
import pandas as pd
import spacy
import string
import re
import csv
import time
import tweepy
import matplotlib.pyplot as plt

from gensim.models.word2vec import Word2Vec
from string import punctuation 

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import label_binarize, MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
punctuations = string.punctuation
#nlp = spacy.load('en_core_web_md')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
APIkey = 'YOURAPIKEY'
APIsecretkey = 'YOURAPISECRETKEY'
accesstokensecret = 'ACCESSTOKENSECRET'
accesstoken = 'ACCESSTOKEN'

twitter_api = twitter.Api(consumer_key=APIkey,consumer_secret=APIsecretkey,access_token_key=accesstoken,access_token_secret=accesstokensecret)


# Search Set

In [ ]:
auth = tweepy.OAuthHandler(APIkey, APIsecretkey )
auth.set_access_token(accesstoken, accesstokensecret)
api = tweepy.API(auth, wait_on_rate_limit=True)

tweets = []

def querytweets(text_query, count):
#tweepy API to query tweets for desired search term
        tweets = tweepy.Cursor(api.search, q=text_query, lang='en', 
                               #geocode=coordinates,
                               ).items(count)
        tweets_list = [[tweet.created_at, tweet.text] for tweet in tweets]
        tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Text'])
        return tweets_df

In [ ]:
text_query = input("Enter a search keyword: ") 

#number of desired tweets here
count = 10
searchsetdf = querytweets(text_query, count)

Enter a search keyword: blackpink


In [ ]:
searchsetdf

,Datetime,Text
0,2020-10-19 06:44:20,RT @blackpinkbabo: JENNIE LIKES THAT! Look at ...
1,2020-10-19 06:44:20,RT @ygent_official: #BLACKPINK 'Kill This Love...
2,2020-10-19 06:44:19,RT @13chnwar: rosie is whipped 😂\n\n#JENNIE #제...
3,2020-10-19 06:44:19,RT @BLACKPINKGLOBAL: RT &amp; REPLY:\n\nI vote...
4,2020-10-19 06:44:19,RT @TODAYonline: Blackpink's Lisa Is M.A.C's F...
5,2020-10-19 06:44:19,RT @jnyloops: hi! this is a new account dedica...
6,2020-10-19 06:44:19,RT @TeamLisaEU: 📌 #LISA: I have always been a ...
7,2020-10-19 06:44:19,RT @BLACKPINKGLOBAL: #JISOO #JENNIE #ROSÉ for ...
8,2020-10-19 06:44:18,RT @BLACKPINK: 'THE ALBUM' JACKET MAKING FILM\...
9,2020-10-19 06:44:18,"RT @BLINKVotingPage: [PCA 2020]\n\nLast year, ..."


# Training Set

In [ ]:
def buildTrainingSet(corpusFile, tweetDataFile):
#use corpusFile which has id keys for tweets to grab from twitter API as to not violate any developer rules
    corpus=[]
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile, delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})

    rate_limit=180
    sleep_time=900/180
    trainingDataSet=[]
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time)
        except: 
            continue
    # Now we write them to the empty CSV file
    with open(tweetDataFile,'wb') as csvfile:
        linewriter=csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"],tweet["text"],tweet["label"],tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [ ]:
#corpusFile = "/YOURFILEPATH/corpus.csv"
#tweetDataFile = "/YOURFILEPATH/tweetDataFile.csv"
#trainingData = buildTrainingSet(corpusFile, tweetDataFile)

In [ ]:
# Once we ran buildTrainingSet once (takes a few hours), now we have csv of all the tweets for training

trainingdata = pd.read_csv("/YOURFILEPATH/full-corpus.csv")

In [ ]:
trainingdata

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...
...,...,...,...,...,...
5108,twitter,irrelevant,126855687060987904,Thu Oct 20 03:02:07 +0000 2011,me re copè con #twitter
5109,twitter,irrelevant,126855171702661120,Thu Oct 20 03:00:04 +0000 2011,Buenas noches genteeee :) #twitter los quieroo...
5110,twitter,irrelevant,126854999442587648,Thu Oct 20 02:59:23 +0000 2011,#twitter tiene la mala costumbre de ponerce bn...
5111,twitter,irrelevant,126854818101858304,Thu Oct 20 02:58:40 +0000 2011,Oi @flaviasansi. Muito bem vinda ao meu #Twitt...


In [ ]:
trainingdata['Sentiment'] = trainingdata['Sentiment'].map({'negative':0,'neutral':1,'positive':2,'irrelevant':4})
trainingdata['Sentiment'].value_counts()

1    2333
4    1689
0     572
2     519
Name: Sentiment, dtype: int64

# Pre-Processing

In [ ]:
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    table = str.maketrans({key: None for key in string.punctuation})

    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        #Convert text to lowercase, strip whitespace and remove personal pronouns
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        #Remove stopwords
        tokens = [tok.translate(table) for tok in tokens if tok not in stopwords ]
        tokens = ' '.join(tokens)
        #Remove extra whitespace
        tokens = ' '.join(tokens.split())
        # remove URLs
        tokens = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tokens)
        # remove usernames
        tokens = re.sub('@[^\s]+', 'AT_USER', tokens)
        # remove the # in #hashtag
        tokens = re.sub(r'#([^\s]+)', r'\1', tokens)
        # remove repeated characters (helloooooooo into hello)
        #tokens = word_tokenize(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [ ]:
trainingdata = trainingdata[['TweetText','Sentiment']]

In [ ]:
trainingdata['data_clean']= cleanup_text(trainingdata['TweetText'],logging=True)

Processed 1000 out of 5113 documents.
Processed 2000 out of 5113 documents.
Processed 3000 out of 5113 documents.
Processed 4000 out of 5113 documents.
Processed 5000 out of 5113 documents.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
trainingdata.isnull().sum()

TweetText     0
Sentiment     0
data_clean    0
dtype: int64

In [ ]:
trainingdata = trainingdata[trainingdata['Sentiment'] < 4]

In [ ]:
train,test = train_test_split(trainingdata,test_size=0.2)  

In [ ]:
test = test.sort_index()
train = train.sort_index()

In [ ]:
searchsetdf['data_clean'] = cleanup_text(searchsetdf['Text'],logging=True)
searchsetdf

,Datetime,Text,data_clean
0,2020-10-19 06:44:20,RT @blackpinkbabo: JENNIE LIKES THAT! Look at ...,rt blackpinkbabo jennie likes look jennie ’s l...
1,2020-10-19 06:44:20,RT @ygent_official: #BLACKPINK 'Kill This Love...,rt ygentofficial blackpink kill love dance pra...
2,2020-10-19 06:44:19,RT @13chnwar: rosie is whipped 😂\n\n#JENNIE #제...,rt 13chnwar rosie whip 😂 jennie 제니 rosé 로제 bla...
3,2020-10-19 06:44:19,RT @BLACKPINKGLOBAL: RT &amp; REPLY:\n\nI vote...,rt blackpinkglobal rt amp reply vote icecream ...
4,2020-10-19 06:44:19,RT @TODAYonline: Blackpink's Lisa Is M.A.C's F...,rt todayonline blackpink s lisa mac s first fe...
5,2020-10-19 06:44:19,RT @jnyloops: hi! this is a new account dedica...,rt jnyloop hi new account dedicate blackpink ’...
6,2020-10-19 06:44:19,RT @TeamLisaEU: 📌 #LISA: I have always been a ...,rt teamlisaeu 📌 lisa always big fan mac like b...
7,2020-10-19 06:44:19,RT @BLACKPINKGLOBAL: #JISOO #JENNIE #ROSÉ for ...,rt blackpinkglobal jisoo jennie rosé adidas or...
8,2020-10-19 06:44:18,RT @BLACKPINK: 'THE ALBUM' JACKET MAKING FILM\...,rt blackpink album jacket making film blackpin...
9,2020-10-19 06:44:18,"RT @BLINKVotingPage: [PCA 2020]\n\nLast year, ...",rt blinkvotingpage pca 2020 last year blackpin...


# AutoML

In [ ]:
automltraining = trainingdata
automltraining = automltraining[['TweetText','Sentiment']]

In [ ]:
automltraining

,TweetText,Sentiment
0,Now all @Apple has to do is get swype on the i...,2
1,@Apple will be adding more carrier support to ...,2
2,Hilarious @youtube video - guy does a duet wit...,2
3,@RIM you made it too easy for me to switch to ...,2
4,I just realized that the reason I got into twi...,2
...,...,...
4537,"@madtruckman 'Modern Day Autograph"", I like th...",1
4538,62 Ways to Use #Twitter for Business: http://t...,1
4539,"Log off #Facebook On #Twitter , But I Think i'...",1
4540,"""#twitter's dumb, I don't like it."" Hush up, ...",1


In [ ]:
automltraining['Sentiment'].value_counts()

1    2333
0     572
2     519
Name: Sentiment, dtype: int64

In [ ]:
#write to csv for automl training data, only need to run once 

#automltraining.to_csv("/content/gdrive/My Drive/Colab Notebooks/portfolio/twitter/automltraining.csv",index=None)

In [ ]:
searchsetexport = searchsetdf['data_clean']

In [ ]:
#export each tweet as separate txt file in folder as per AutoML batchpredict file input requirements

for key in searchsetdf.index.unique(): 
    searchsetexport = searchsetdf[searchsetdf.index == key]   
    searchsetexport['data_clean'].to_csv("/content/gdrive/My Drive/Colab Notebooks/portfolio/twitter/tweets/%s.txt" % key, header=False)  

In [ ]:
#write a CSV of each txt file name as reference as per AutoML batchpredict file input requirements

tweetlist = []
for i in searchsetdf.index.unique():
  tweetlist.append("gs://YOURBUCKETURI/%i.txt" % i)
  df = pd.DataFrame(tweetlist)
  df.to_csv("/YOURFILEPATH/tweetlist.csv",index=None,header=None)


# Feature Selection

## bagofwords 

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train['data_clean'])
X_train_counts = X_train_counts.toarray()

X_test_counts = count_vect.transform(test['data_clean']).toarray()

In [ ]:
searchset_counts = count_vect.fit_transform(searchsetdf['data_clean'])
searchset_counts = searchset_counts.toarray()

## tfidf

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf = X_train_tfidf.toarray()

X_test_tfidf = tfidf_transformer.transform(X_test_counts).toarray()

In [ ]:
searchset_tfidf = tfidf_transformer.fit_transform(searchset_counts)
searchset_tfidf = searchset_tfidf.toarray()

searchset_tfidf = tfidf_transformer.transform(searchset_counts).toarray()
searchset_tfidf.shape

(10, 98)

## glove

In [ ]:
train_vec_glove = []
counter = 0
for doc in nlp.pipe(train["data_clean"], batch_size=500):
    if counter % 500 == 0:
        print("Processed %d out of %d documents." % (counter, len(train["data_clean"])))
    if doc.has_vector:
        train_vec_glove.append(doc.vector)
    else:
        train_vec_glove.append(np.zeros((300,), dtype="float32"))
    counter +=1
        
train_vec_glove = np.array(train_vec_glove)

counter = 0
test_vec_glove = []
for doc in nlp.pipe(test["data_clean"], batch_size=500):
    if counter % 1000 == 0:
        print("Processed %d out of %d documents." % (counter, len(test["data_clean"])))
    if doc.has_vector:
        test_vec_glove.append(doc.vector)
    else:# If doc doesn't have a vector, then fill it with zeros.
        test_vec_glove.append(np.zeros((300,), dtype="float32"))
    counter +=1
     
test_vec_glove = np.array(test_vec_glove)

Processed 0 out of 2739 documents.
Processed 500 out of 2739 documents.
Processed 1000 out of 2739 documents.
Processed 1500 out of 2739 documents.
Processed 2000 out of 2739 documents.
Processed 2500 out of 2739 documents.
Processed 0 out of 685 documents.


In [ ]:
print("Train word vector shape:", train_vec_glove.shape)
print("Test word vector shape:", test_vec_glove.shape)

Train word vector shape: (2739, 96)
Test word vector shape: (685, 96)


In [ ]:
%%time
search_vec_glove = []
counter = 0
for doc in nlp.pipe(searchsetdf["data_clean"], batch_size=500):
    if counter % 500 == 0:
        print("Processed %d out of %d documents." % (counter, len(searchsetdf["data_clean"])))
    if doc.has_vector:
        search_vec_glove.append(doc.vector)
    else:
        search_vec_glove.append(np.zeros((300,), dtype="float32"))
    counter +=1
    
search_vec_glove = np.array(search_vec_glove)

Processed 0 out of 10 documents.
CPU times: user 35.7 ms, sys: 3.07 ms, total: 38.8 ms
Wall time: 38.3 ms


In [ ]:
print("Search word vector shape:", search_vec_glove.shape)

Search word vector shape: (10, 96)


# Classifier


## Keras Neural Network

In [ ]:
# Transform labels into one hot encoded format.

y_train_one = label_binarize(train["Sentiment"], classes=train["Sentiment"].unique())

### Glove NN

In [ ]:
# Define number of epochs

epochs = 100
ann_layers = 1
#Standard fully connected network

K.clear_session()

model = Sequential()
model.add(Dense(512, activation='relu', kernel_initializer='he_normal', input_dim=96))
model.add(Dropout(0.3))

for i in range(ann_layers):
    model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.3))

model.add(Dense(3, activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['acc'])

early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

# Fit the model to the training data
ann_glove = model.fit(train_vec_glove, y_train_one, epochs=epochs,validation_split=0.1, verbose=1,
                      callbacks=[early_stop])


Epoch 1/100
78/78 [==============================] - 0s 6ms/step - loss: 1.2118 - acc: 0.5907 - val_loss: 0.4764 - val_acc: 0.9599
Epoch 2/100
78/78 [==============================] - 0s 4ms/step - loss: 0.8945 - acc: 0.6227 - val_loss: 0.4895 - val_acc: 0.9635
Epoch 3/100
78/78 [==============================] - 0s 4ms/step - loss: 0.8603 - acc: 0.6450 - val_loss: 1.0694 - val_acc: 0.3394
Epoch 4/100
78/78 [==============================] - 0s 4ms/step - loss: 0.8650 - acc: 0.6389 - val_loss: 0.9931 - val_acc: 0.4343
Epoch 5/100
78/78 [==============================] - 0s 4ms/step - loss: 0.8683 - acc: 0.6264 - val_loss: 0.6621 - val_acc: 0.8540
Epoch 6/100
78/78 [==============================] - 0s 4ms/step - loss: 0.8381 - acc: 0.6345 - val_loss: 0.6093 - val_acc: 0.9234
Epoch 7/100
78/78 [==============================] - 0s 4ms/step - loss: 0.8267 - acc: 0.6483 - val_loss: 0.3979 - val_acc: 0.9964
Epoch 8/100
78/78 [==============================] - 0s 5ms/step - loss: 0.7846 - a

In [ ]:
print(f"Training accuracy: {100*ann_glove.history['acc'][-1]}")
print(f"Validation accuracy: {100*ann_glove.history['val_acc'][-1]} for the Glove embeddings.")

Training accuracy: 87.22109794616699
Validation accuracy: 81.38686418533325 for the Glove embeddings.


In [ ]:
#Predict class labels on search set

searchlabelsglove = np.argmax(model.predict(search_vec_glove), axis=-1)
searchsetdf['glovelabel'] = searchlabelsglove

print(f'Percentage of Negative(0), Neutral(1), and Positive(2) Tweets for keyword: {text_query} from a total of {count} tweets')
searchsetdf['glovelabel'].value_counts(normalize=True,sort=True) * 100

Percentage of Negative(0), Neutral(1), and Positive(2) Tweets for keyword: blackpink 


2    80.0
1    10.0
0    10.0
Name: glovelabel, dtype: float64